## 1. Import libraries

In [1]:
from glob import glob
import cv2
import rasterio
import numpy as np
import random
import os
import torch
import pandas as pd
from tqdm import tqdm
import joblib


## 2. Config

In [40]:
config={
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'seed':10,
    'val_size':0.2,
    'experiment':True,
    'batch_size':8,
    'lr':1e-3,
    'epochs':50,
    'early_stopping':10,
    'inference':{
        'threshold':0.8
    }
}
DATAPATH='./sm_dataset'
data_paths={
    'train_csv': DATAPATH+'/sm_npy_train_meta.csv' if config['experiment'] else DATAPATH+'/train_meta.csv',
    'test_csv': DATAPATH+'/test_meta.csv',
    'train_img':DATAPATH+'/npy_train_img/',
    'train_mask':DATAPATH+'/npy_train_mask/'
}
train_csv = pd.read_csv(data_paths['train_csv'])
test_csv = pd.read_csv(data_paths['test_csv'])
SAVE_PATH = './logs'
save_paths={
    'model':SAVE_PATH+'/models/',
    'val':SAVE_PATH+'/vals/'
}
# 시드 고정 함수
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
fix_seed(config['seed'])

## 3. Load Data

In [3]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms

In [22]:
class Wildfire_Dataset(Dataset):
    def __init__(self, csv, img_transform=None,mask_transform = None,infer=False):
        self.csv = csv
        self.img_transform = img_transform
        self.mask_transform = mask_transform
        self.infer = infer
    def __len__(self):
        return len(self.csv)
    
    def __getitem__(self,idx):
        img_path = self.csv.iloc[idx,0]
        image = np.load(data_paths['train_img']+img_path).transpose(2,0,1)
        # test인 경우 infer==True
        if self.infer:
            if self.img_transform:
                for i in range(10):
                    image[i] = self.img_transform(image)
            return image
        
        mask_path = self.csv.iloc[idx,1]
        mask = np.load(data_paths['train_mask']+mask_path)
        if self.img_transform:
            for i in range(10):
                image[i] = self.img_transform(image[i])
            mask = torch.from_numpy(self.mask_transform(mask))
            
        return image, mask

In [10]:
mask_transforms = {
    'train': transforms.Compose([
        # transforms.ToTensor(),
    ]),
    'valid':transforms.Compose([
        # transforms.ToTensor(),
    ]),
    'test':transforms.Compose([
        # transforms.ToTensor(),
    ]),
}
img_transforms={
    'train': transforms.Compose([
        mask_transforms['train'],
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, )),
    ]),
    'valid':transforms.Compose([
        mask_transforms['valid'],
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, )),
    ]),
    'test':transforms.Compose([
        mask_transforms['valid'],
        transforms.Normalize((0.5, ), (0.5, )),
    ]),
}

In [22]:
mask = np.load(data_paths['train_mask']+'/train_mask_0.npy')

In [25]:
tr = transforms.ToTensor()
tr(mask).dtype

torch.float32

In [23]:
train_csv_, valid_csv_ = train_test_split(train_csv,test_size=config['val_size'],random_state=config['seed'])
train_dataset = Wildfire_Dataset(train_csv_,img_transform=img_transforms['train'],mask_transform=mask_transforms['train'],infer=False)
valid_dataset = Wildfire_Dataset(valid_csv_,img_transform=img_transforms['valid'],mask_transform=mask_transforms['valid'],infer=False)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'],shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'],shuffle=False)
print(f'train size: {len(train_dataset)}, valid size: {len(valid_dataset)}')

train size: 16, valid size: 4


In [15]:
train_dataset[0][1]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.uint8)

## 4. Modeling

In [16]:
import segmentation_models_pytorch as smp

C:\Users\artis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
model = smp.Unet(
    encoder_name='efficientnet-b7',
    encoder_weights = 'imagenet',
    in_channels=10,
    classes=1,
)
torch.save(model.state_dict(), save_paths["model"]+'/base_model.pt')


## 5. Train/Validation

In [43]:
def validation(config, model, criterion, valid_loader,epoch):
    model.eval()
    valid_loss=0
    y_pred_dict={}
    with torch.no_grad():
        for idx,(images, masks) in tqdm(enumerate(valid_loader)):
            images = images.to(config['device'],dtype = torch.float32)
            masks = masks.to(config['device'],dtype = torch.float32)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            valid_loss +=loss.item()
            
            output_masks = torch.sigmoid(outputs).cpu().numpy()
            output_masks = np.squeeze(output_masks, axis=1)
            
            for i in range(len(output_masks)):
                y_pred = np.where(output_masks[i,:,:]>config['inference']['threshold'],1,0).astype(np.uint8)
                y_pred_dict[idx*config['batch_size']+i] = y_pred
        joblib.dump(y_pred_dict,save_paths['val']+f'epoch: {epoch}.pkl')
    return valid_loss/len(valid_loader)

In [19]:
import torch.nn as nn

In [26]:
def train(config, model, train_loader, valid_loader):
    model = model.to(config['device'])
    es_count = 0
    min_val_loss = float('inf')
    best_model = None
    
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr = config['lr'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-8, verbose=True)
    print('***TRAINING START***')
    for epoch in range(config['epochs']):
        model.train()
        epoch_loss=0
        for images, masks in tqdm(train_loader):
            images = images.to(config['device'],dtype = torch.float32)
            masks = masks.to(config['device'],dtype = torch.float32)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            epoch_loss+=loss.item()
        
        val_loss = validation(config, model, criterion, valid_loader, epoch)
        
        es_count += 1
        if min_val_loss > val_loss:
            es_count = 0
            min_val_loss = val_loss
            best_model = model
            state_dict = model.state_dict()
            best_epoch = epoch+1
            print(f"Epoch [{best_epoch}] New Minimum Valid Loss!")
            print("..save current best model..")
            model_name = f'epoch {epoch}_current_best_model.pt'
            torch.save(state_dict, save_paths['model']+'/'+model_name)
        
        scheduler.step(val_loss)
        
        if es_count == config['early_stopping']:
            print(f"Early Stopping Count에 도달했습니다!")
            print(f"Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, Best Epoch: {best_epoch}")
            print("***TRAINING DONE***")
            return best_model
        print(f"Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, ES Count: {es_count}")
        print("------------------------------------------------------------------------------------")
    print(f"Early Stopping Count에 도달하지 않았습니다! \nEarly Stopping Count: {config['early_stopping']} Best Epoch: {best_epoch}")
    print("***TRAINING DONE***")
    return best_model

In [ ]:
best_model = train(config,model,train_loader,valid_loader)

## 6. Inference

In [ ]:
y_pred_dict = {}

for i in test_csv['test_img']:
    y_pred = model.predict(np.array([img]), batch_size=1)

    y_pred = np.where(y_pred[0, :, :, 0] > 0.25, 1, 0) # 임계값 처리
    y_pred = y_pred.astype(np.uint8)
    y_pred_dict[i] = y_pred

joblib.dump(y_pred_dict, './y_pred.pkl')

## 7. Validation Viewer